In [ ]:
!pip install nibabel

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr, wilcoxon
from statsmodels.stats.multitest import fdrcorrection
from itertools import product
import nibabel as nib

# Load AAL NIfTI and XML
aal1_img = nib.load("AAl.nii")
aal1_labels = aal1_img.get_fdata()

aal1_names = pd.read_xml("AAL.xml", xpath=".//label")
aal1_names["index"] = pd.to_numeric(aal1_names["index"], errors="coerce")

# Remove undefined regions
remove_indices = list(map(float, aal1_names["index"].iloc[90:].tolist())) + [0.0]


In [ ]:
import pandas as pd
import numpy as np

# Define file paths for each frequency band
shap_files = {
    "Alpha": "C:/Users/gerar/Downloads/transposed_shap_files/Alpha_rf_feature_importance_reformatted.csv",
    "Beta": "C:/Users/gerar/Downloads/transposed_shap_files/Beta_rf_feature_importance_reformatted.csv",
    "Delta": "C:/Users/gerar/Downloads/transposed_shap_files/Delta_rf_feature_importance_reformatted.csv",
    "Gamma1": "C:/Users/gerar/Downloads/transposed_shap_files/Gamma1_rf_feature_importance_reformatted.csv",
    "Gamma2": "C:/Users/gerar/Downloads/transposed_shap_files/Gamma2_rf_feature_importance_reformatted.csv",
    "Theta": "C:/Users/gerar/Downloads/transposed_shap_files/Theta_rf_feature_importance_reformatted.csv",
}

band_values = []
brain_regions = None

# Read and store SHAP values
for band, path in shap_files.items():
    df = pd.read_csv(path, header=None)
    if brain_regions is None:
        brain_regions = df.iloc[0].tolist()  # First row: brain region names
    band_values.append(df.iloc[1].values.astype(float))  # Second row: SHAP values

# Create final DataFrame (rows=frequency bands, columns = brain regions)
shap_df = pd.DataFrame(
    data=np.array(band_values),
    index=[f"{band}_logreg_mean_abs_shap" for band in shap_files.keys()],
    columns=brain_regions
)

print(shap_df.head())
df


                             Precentral_L  Precentral_R  Frontal_Sup_L  \
Alpha_logreg_mean_abs_shap       0.018404      0.011823       0.022591   
Beta_logreg_mean_abs_shap        0.003386      0.003786       0.003228   
Delta_logreg_mean_abs_shap       0.008149      0.008141       0.007784   
Gamma1_logreg_mean_abs_shap      0.007422      0.010436       0.018565   
Gamma2_logreg_mean_abs_shap      0.008364      0.012214       0.021049   

                             Frontal_Sup_R  Frontal_Sup_Orb_L  \
Alpha_logreg_mean_abs_shap        0.019229           0.004753   
Beta_logreg_mean_abs_shap         0.017457           0.022040   
Delta_logreg_mean_abs_shap        0.030430           0.012329   
Gamma1_logreg_mean_abs_shap       0.006374           0.016761   
Gamma2_logreg_mean_abs_shap       0.025030           0.007245   

                             Frontal_Sup_Orb_R  Frontal_Mid_L  Frontal_Mid_R  \
Alpha_logreg_mean_abs_shap            0.025857       0.010410       0.009480   
Beta

,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
0,Precentral_L,Precentral_R,Frontal_Sup_L,Frontal_Sup_R,Frontal_Sup_Orb_L,Frontal_Sup_Orb_R,Frontal_Mid_L,Frontal_Mid_R,Frontal_Mid_Orb_L,Frontal_Mid_Orb_R,...,Temporal_Sup_L,Temporal_Sup_R,Temporal_Pole_Sup_L,Temporal_Pole_Sup_R,Temporal_Mid_L,Temporal_Mid_R,Temporal_Pole_Mid_L,Temporal_Pole_Mid_R,Temporal_Inf_L,Temporal_Inf_R
1,0.00493750402728,0.0083748001210592,0.0125227384738374,0.0144355600836773,0.0030003267495173,0.0145159622316273,0.0137966462561684,0.008497210176736,0.0089596920312734,0.0115438782246566,...,0.0218304871114452,0.010347740147354,0.0099173323973819,0.0063239780453132,0.0149964263358161,0.0327226870073947,0.0071422219745136,0.0050524658185766,0.0247921008195449,0.0304960640485639


In [ ]:
from tqdm import tqdm
import numpy as np
import nibabel as nib

# Create a dictionary to map index values to region names
index_to_name = dict(zip(aal1_names["index"], aal1_names["name"]))

# EEG bands (must match the index names in shap_df)
bands = ["Alpha", "Beta", "Delta", "Gamma1", "Gamma2", "Theta"]

for band in bands:
    print(f"Processing {band}...")
    new_data = np.zeros((91, 109, 91), dtype=np.float32)

    for i in tqdm(range(91), desc=f"{band} - i"):
        for j in range(109):
            for k in range(91):
                idx = float(aal1_labels[i, j, k])
                if idx not in remove_indices:
                    region_name = index_to_name.get(idx)
                    if region_name and region_name in shap_df.columns:
                        value = shap_df.loc[f"{band}_logreg_mean_abs_shap", region_name]
                        new_data[i, j, k] = value

    # Save to NIfTI
    out_img = nib.Nifti1Image(new_data, affine=aal1_img.affine, header=aal1_img.header)
    nib.save(out_img, f"C:/Users/gerar/Downloads/{band}.nii.gz")
    print(f"✅ Saved /content/{band}.nii.gz")


Processing Alpha...


Alpha - i: 100%|██████████| 91/91 [00:01<00:00, 45.85it/s] 


✅ Saved /content/Alpha.nii.gz
Processing Beta...


Beta - i: 100%|██████████| 91/91 [00:01<00:00, 46.15it/s] 


✅ Saved /content/Beta.nii.gz
Processing Delta...


...

Gamma2 - i: 100%|██████████| 91/91 [00:02<00:00, 45.16it/s] 


✅ Saved /content/Gamma2.nii.gz
Processing Theta...


Theta - i: 100%|██████████| 91/91 [00:02<00:00, 45.20it/s] 

✅ Saved /content/Theta.nii.gz


: 